In [1]:
import pyomo as pyo
from pyomo.environ import *
import numpy as np
from datetime import datetime, timedelta
import pandas as pd
from shift_and_coverage import create_shifts, is_shift_covering_period

In [13]:
employee_count = 8
employee_ids = ['emp{}'.format(i) for i in range(employee_count)]
employee_ids

['emp0', 'emp1', 'emp2', 'emp3', 'emp4', 'emp5', 'emp6', 'emp7']

### Set of days in the planning horizon & Time periods
**Intervals_in_horizon is the Set of time periods 1....48 intervals in 30 min**
**days in the horizon are the Set of days in the planning horizon**

In [14]:
weeks_in_horizon = 1 #weeks
horizon_lenght = 2 * weeks_in_horizon # days in horizon
period_length = 60 # interval definition
day_length = 24 * 60 // period_length    # intervals
intervals_in_horizon = list(range(day_length)) # intervals in horizon

days_in_horizon = list(range(horizon_lenght))
day_length

24

### Create demand for each time period

In [15]:
demand = {
    0: 4,
    1: 3,
    2: 3,
    3: 2 ,
    4: 1,
    5: 1,
    6: 1,
    7: 1,
    8: 1,
    9: 1,
    10: 2,
    11: 2,
    12: 2,
    13: 2,
    14: 1,
    15: 1,
    16: 2,
    17: 2,
    18: 2,
    19: 2,
    20: 2,
    21: 2,
    22: 3,
    23: 4,
    }

In [16]:
""" demand = {
    0: {hour: demand_value for hour, demand_value in enumerate([4, 3, 3, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 3, 4], start=1)},
    1: {hour: demand_value for hour, demand_value in enumerate([4, 3, 3, 2, 1, 1, 1, 1, 1, 3, 3, 3, 3, 1, 1, 2, 2, 2, 2, 2, 2, 3, 5], start=1)}
}
print("Demand structure:", demand)
 """

' demand = {\n    0: {hour: demand_value for hour, demand_value in enumerate([4, 3, 3, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 3, 4], start=1)},\n    1: {hour: demand_value for hour, demand_value in enumerate([4, 3, 3, 2, 1, 1, 1, 1, 1, 3, 3, 3, 3, 1, 1, 2, 2, 2, 2, 2, 2, 3, 5], start=1)}\n}\nprint("Demand structure:", demand)\n '

### Shift creation, create possible shifts with min and max duration

In [17]:
shifts =create_shifts(60)
shifts

[(7, 6),
 (7, 7),
 (7, 8),
 (7, 9),
 (8, 6),
 (8, 7),
 (8, 8),
 (8, 9),
 (9, 6),
 (9, 7),
 (9, 8),
 (9, 9),
 (10, 6),
 (10, 7),
 (10, 8),
 (10, 9),
 (11, 6),
 (11, 7),
 (11, 8),
 (11, 9),
 (12, 6),
 (12, 7),
 (12, 8),
 (12, 9),
 (13, 6),
 (13, 7),
 (13, 8),
 (13, 9),
 (14, 6),
 (14, 7),
 (14, 8),
 (14, 9),
 (15, 6),
 (15, 7),
 (15, 8),
 (15, 9),
 (16, 6),
 (16, 7),
 (16, 8),
 (16, 9),
 (17, 6),
 (17, 7),
 (17, 8),
 (17, 9),
 (18, 6),
 (18, 7),
 (18, 8),
 (18, 9),
 (19, 6),
 (19, 7),
 (19, 8),
 (19, 9),
 (20, 6),
 (20, 7),
 (20, 8),
 (20, 9),
 (21, 6),
 (21, 7),
 (21, 8),
 (21, 9),
 (22, 6),
 (22, 7),
 (22, 8),
 (22, 9),
 (23, 6),
 (23, 7),
 (23, 8),
 (23, 9),
 (0, 6),
 (0, 7),
 (0, 8),
 (0, 9)]

### Pyomo Model

In [18]:
# Define the Pyomo model
model = ConcreteModel()

# Sets
model.E = Set(initialize=employee_ids)  # Set of employee IDs
model.T = Set(initialize=intervals_in_horizon)  # Set of time periods
#model.S = Set(initialize=create_shifts(period_length))  # Set of possible shifts
# Validation: 
Shifts = [(6,10), (14,8), (21,10)]
model.S = Set(initialize=Shifts)
#model.D = Set(initialize=days_in_horizon)

# Variables
model.x = Var(model.E, model.S, domain=Binary)        # Employees assigned to work shift s 


In [19]:
# Print
print("Employees (E):", model.E.pprint())
print("Time Periods (T):", model.T.pprint())
print("Shifts (S):", model.S.pprint())

E : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     1 :    Any :    8 : {'emp0', 'emp1', 'emp2', 'emp3', 'emp4', 'emp5', 'emp6', 'emp7'}
Employees (E): None
T : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     1 :    Any :   24 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23}
Time Periods (T): None
S : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     2 :    Any :    3 : {(6, 10), (14, 8), (21, 10)}
Shifts (S): None


In [20]:

# Constraints
def demand_constraint_rule(model, time):
    if time in model.T: 
        return sum(model.x[employee, shift] for employee in model.E for shift in model.S if is_shift_covering_period(shift, time, day_length)) >= demand[time]
    else:
            # If demand for this period is not defined, no constraint is needed
        return Constraint.Skip
    
model.demand_constraint = Constraint(model.T, rule=demand_constraint_rule)

def exclusive_shifts_constraint(model, employee, time):
    return sum(model.x[employee, shift] for shift in model.S if is_shift_covering_period(shift, time, day_length)) <= 1

model.exclusive_shifts_constraint = Constraint(model.E, model.T, rule=exclusive_shifts_constraint)

model.obj = Objective(expr=1, sense=minimize)

# Solve the model
solver = SolverFactory('gurobi')
results = solver.solve(model)

# Print results
if str(results.solver.termination_condition) == "optimal":
    print("Solution found:")
    for employee in model.E:
        for shift in model.S:
            if value(model.x[employee, shift]):
                print(f"Assign {employee} to shift starting at {shift[0]} with duration {shift[1]}")
else:
    print("No feasible solution found.")


Solution found:
Assign emp0 to shift starting at 6 with duration 10
Assign emp1 to shift starting at 21 with duration 10
Assign emp2 to shift starting at 14 with duration 8
Assign emp3 to shift starting at 21 with duration 10
Assign emp4 to shift starting at 21 with duration 10
Assign emp5 to shift starting at 21 with duration 10
Assign emp6 to shift starting at 6 with duration 10
Assign emp7 to shift starting at 14 with duration 8


In [11]:
model.T.display()

T : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     1 :    Any :   24 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23}


In [12]:
model.x.display()

x : Size=21, Index=x_index
    Key              : Lower : Value : Upper : Fixed : Stale : Domain
     ('emp0', 6, 10) :     0 :  None :     1 : False :  True : Binary
     ('emp0', 14, 8) :     0 :  None :     1 : False :  True : Binary
    ('emp0', 21, 10) :     0 :  None :     1 : False :  True : Binary
     ('emp1', 6, 10) :     0 :  None :     1 : False :  True : Binary
     ('emp1', 14, 8) :     0 :  None :     1 : False :  True : Binary
    ('emp1', 21, 10) :     0 :  None :     1 : False :  True : Binary
     ('emp2', 6, 10) :     0 :  None :     1 : False :  True : Binary
     ('emp2', 14, 8) :     0 :  None :     1 : False :  True : Binary
    ('emp2', 21, 10) :     0 :  None :     1 : False :  True : Binary
     ('emp3', 6, 10) :     0 :  None :     1 : False :  True : Binary
     ('emp3', 14, 8) :     0 :  None :     1 : False :  True : Binary
    ('emp3', 21, 10) :     0 :  None :     1 : False :  True : Binary
     ('emp4', 6, 10) :     0 :  None :     1 : False :  True : 

In [ ]:
""" def exclusive_shifts_constraint(model, employee, time):
    # Employee 'e' cannot work two shifts simultaneously
    return sum(model.x[employee, shift] for shift in model.S if is_shift_covering_period(shift, time, day_length)) <= 1

model.exclusive_shifts_constraint = Constraint(model.E, model.T, rule=exclusive_shifts_constraint) """
